In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
from copy import copy
import numpy as np
import pandas as pd
import operator as op
from scipy.stats import logistic
from scipy.stats import entropy
from numpy import (
    arctanh,
    tanh,
    pi,
)
from math import sqrt

# get connected to the database, dont forget to change the password
connection = pg.connect("dbname=aligulac user=postgres password=*----------- CHANGE TO YOUR PASSWORD -----------*")
query = 'SELECT match.id, match.pla_id, match.plb_id, match.sca, match.scb, ratinga.rating as "ra", ratinga.rating_vp as "ra_vp", ratinga.rating_vt as "ra_vt", ratinga.rating_vz as "ra_vz", ratingb.rating as "rb", ratingb.rating_vp as "rb_vp", ratingb.rating_vt as "rb_vt", ratingb.rating_vz as "rb_vz" FROM "match" match, "rating" ratinga, "rating" ratingb WHERE match.rta_id = ratinga.id AND match.rtb_id = ratingb.id'
dataframe = psql.read_sql_query(query, connection)
dataframe[:10]

,id,pla_id,plb_id,sca,scb,ra,ra_vp,ra_vt,ra_vz,rb,rb_vp,rb_vt,rb_vz
0,132732,324,438,0,1,0.005730,0.027205,-0.016116,-0.011089,-0.012237,0.079859,-0.057182,-0.022677
1,206034,1100,163,1,3,0.573410,-0.077954,-0.031185,0.109139,0.356222,-0.006268,-0.065864,0.072133
2,210461,5602,317,1,0,0.630058,-0.042661,-0.073657,0.116318,0.879652,-0.025865,-0.019010,0.044875
3,205617,10953,362,2,0,0.307009,-0.039786,0.020636,0.019149,0.367283,0.003825,-0.055976,0.052151
4,32420,493,555,1,2,-0.076174,0.110510,-0.097360,-0.013150,-0.196227,0.020901,-0.031547,0.010647
5,64934,270,416,1,0,0.192390,0.046838,-0.044708,-0.002131,0.136472,-0.112990,0.095361,0.017629
6,76695,182,1016,2,3,0.219193,0.059885,-0.122530,0.062645,0.103270,0.034849,-0.096764,0.061914
7,137781,41,72,2,1,0.724542,-0.089416,0.071528,0.017888,0.580553,0.208396,-0.165082,-0.043314
8,23316,671,1223,2,0,0.033435,-0.060389,-0.053444,0.113833,-0.010314,0.000000,0.000000,0.000000
9,38135,442,151,2,1,-0.053839,0.026228,-0.026228,0.000000,0.284978,0.023533,0.039111,-0.062644


In [2]:
#Identify the graphs on the complete set and prepare the matches dictionary

players = {}
matches = {}
beat = {}
is_beated = {}

intransitivities = 0
transitivities = 0

for index, row in dataframe.iterrows():
    players[row['pla_id']]=players.get(row['pla_id'], 0)+1
    players[row['plb_id']]=players.get(row['plb_id'], 0)+1
    
    player_a = min(row['pla_id'], row['plb_id'])
    player_b = max(row['pla_id'], row['plb_id'])
    matches[(player_a, player_b)]=matches.get((player_a, player_b), 0)+1
    
    winner = row['pla_id'] if row['sca'] > row['scb'] else row['plb_id']
    loser = row['pla_id'] if row['sca'] < row['scb'] else row['plb_id']
    
    winner_beats = beat.get(winner, set())
    winner_beats.add(loser)
    beat[winner] = winner_beats
    loser_beaten = is_beated.get(loser, set())
    loser_beaten.add(winner)
    is_beated[loser] = loser_beaten
    
    loser_beats = beat.get(loser, set())
    winner_beaten = is_beated.get(winner, set())
    
    intransitivities += len(loser_beats & winner_beaten)
    transitivities += len(winner_beats & loser_beaten) + len(winner_beats & loser_beats) + len(winner_beaten & loser_beaten)
    
print intransitivities
print transitivities

2460735
9884526


In [3]:
#filter complete set to reduced set

perfDF = dataframe.copy()
to_delete = []
for index, row in perfDF.iterrows():
    player_a = min(row['pla_id'], row['plb_id'])
    player_b = max(row['pla_id'], row['plb_id'])
    if matches[(player_a, player_b)] < 30:
        to_delete.append(index)

perfDF.drop(perfDF.index[to_delete], inplace=True)
perfDF.reset_index(drop=True, inplace=True)

In [4]:
#calculation of KL-Divergence on reduced set

expected = {}
observed = {}
kl_divergence = []
for index, row in perfDF.iterrows():
    player_a = min(row['pla_id'], row['plb_id'])
    player_b = max(row['pla_id'], row['plb_id'])
    key = (player_a, player_b)
    
    adj_a = (row['ra_vp']+row['ra_vt']+row['ra_vz'])/3
    rating_a = adj_a + row['ra']
    adj_b = (row['rb_vp']+row['rb_vt']+row['rb_vz'])/3
    rating_b = adj_b + row['rb']
    
    a_rat,b_rat = (rating_a,rating_b) if row['pla_id'] == player_a else (rating_b,rating_a)
    a_score,b_score = (row['sca'],row['scb']) if row['pla_id'] == player_a else (row['scb'],row['sca'])
    expected[key]=expected.get(key, 0) + logistic.cdf(a_rat - b_rat)
    observed[key]=observed.get(key, 0) + (1 if a_score > b_score else 0)

for key in expected:
    expected[key] /= matches[key]
    observed[key] /= float(matches[key])
    exp = [expected[key], 1-expected[key]]
    obs = [observed[key], 1-observed[key]]
    kl_divergence.append(entropy(exp,obs))
    
kl_divergence
sum(kl_divergence)/len(kl_divergence)

0.052765963138623917

In [5]:
#Identify the graphs on the reduced set

players = {}
matches = {}
beat = {}
is_beated = {}

intransitivities = 0
transitivities = 0

for index, row in perfDF.iterrows():
    players[row['pla_id']]=players.get(row['pla_id'], 0)+1
    players[row['plb_id']]=players.get(row['plb_id'], 0)+1
    
    player_a = min(row['pla_id'], row['plb_id'])
    player_b = max(row['pla_id'], row['plb_id'])
    matches[(player_a, player_b)]=matches.get((player_a, player_b), 0)+1
    
    winner = row['pla_id'] if row['sca'] > row['scb'] else row['plb_id']
    loser = row['pla_id'] if row['sca'] < row['scb'] else row['plb_id']
    
    winner_beats = beat.get(winner, set())
    winner_beats.add(loser)
    beat[winner] = winner_beats
    loser_beaten = is_beated.get(loser, set())
    loser_beaten.add(winner)
    is_beated[loser] = loser_beaten
    
    loser_beats = beat.get(loser, set())
    winner_beaten = is_beated.get(winner, set())
    
    intransitivities += len(loser_beats & winner_beaten)
    transitivities += len(winner_beats & loser_beaten) + len(winner_beats & loser_beats) + len(winner_beaten & loser_beaten)
    
print intransitivities
print transitivities
print len(players)

2081
6262
41


In [6]:
#Description of the dictionary with matches frequencies

match_df = pd.DataFrame.from_dict(matches, orient='index')
match_df.describe()

,0
count,49.000000
mean,36.734694
std,7.702206
min,30.000000
25%,31.000000
50%,33.000000
75%,41.000000
max,56.000000


In [7]:
#Transitivity index with the reduced set

def ncr(n, r):
    r = min(r, n-r)
    if r == 0: return 1
    numer = reduce(op.mul, xrange(n, n-r, -1))
    denom = reduce(op.mul, xrange(1, r+1))
    return numer//denom

float(intransitivities) / ncr(len(players), 3)

0.1952157598499062